In [18]:
import numpy as np
import cv2
import sys


In [19]:
view1 = cv2.imread('C:/Documents-Summer2018/CVIP/PA2/view1.png',0)
view5 = cv2.imread('C:/Documents-Summer2018/CVIP/PA2/view5.png',0)

disp1 = cv2.imread('C:/Documents-Summer2018/CVIP/PA2/disp1.png',0)
disp5 = cv2.imread('C:/Documents-Summer2018/CVIP/PA2/disp5.png',0)

# view1 = np.pad(view1,[(1,1),(1,1)], mode="constant")
# view5 = np.pad(view5,[(1,1),(1,1)], mode="constant")
# disp1 = np.pad(disp1,[(1,1),(1,1)], mode="constant")
# disp5 = np.pad(disp5,[(1,1),(1,1)], mode="constant")


row = int(view1.shape[0])
col = int(view1.shape[1])

disp_1 = np.zeros((row,col));
disp_5 = np.zeros((row,col));

#print(view1.shape,view5.shape,disp1.shape,disp5.shape)
#print(disp_1.shape,disp_5.shape)

#print(view1)
#print(view5)
#print(disp1)

In [4]:
# for x in range(1,371):     
#     for y in range(1,464):
#         block1 = view1[x-1:x+2,y-1:y+2] #block for view1
#         min_dist = sys.maxsize    #max integer value 
#         for z in range(y,464):
#             block2 = view5[x-1:x+2,z-1:z+2]
#             result = np.sum((np.subtract(block1,block2)) ** 2)                
#             if(result < min_dist):
#                 min_dist = result
#                 match_y = block2                
#         disp_1[x-1:x+2,y-1:y+2] = match_y - block1  



In [20]:
for x in range(1,row-1):     
    for y in range(1,col-1):
        block1 = view1[x-1:x+2,y-1:y+2] #block for view1
        min_dist = sys.maxsize    #max integer value 
        for z in range(y-100,y):#threshold : 100px
            if(z < 1):
                z = 1
            k = z
            block2 = view5[x-1:x+2,k-1:k+2]
            result = np.sum((np.subtract(block1,block2)) ** 2)
            
            if(result < min_dist):
                min_dist = result
                match_y = k 
                
        disp_1[x,y] = np.absolute(y - match_y) 
        

In [21]:
print(disp_1)

MSE_left = np.mean(np.square(disp1 - disp_1))#https://stackoverflow.com/questions/16774849/mean-squared-error-in-numpy
print("MSE Left:",MSE_left)


cv2.imshow('disp1_norm', disp_1/255)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imwrite('C:/Documents-Summer2018/CVIP/PA2/OPTnew/disparity_left_3x3.jpg',disp_1)

print('disp_1',disp_1)


[[ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  1. ... 30. 32.  0.]
 [ 0.  0.  1. ... 30. 32.  0.]
 ...
 [ 0.  0.  1. ... 82. 72.  0.]
 [ 0.  0.  1. ... 82. 82.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]]
MSE Left: 511.4610413869593
disp_1 [[ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  1. ... 30. 32.  0.]
 [ 0.  0.  1. ... 30. 32.  0.]
 ...
 [ 0.  0.  1. ... 82. 72.  0.]
 [ 0.  0.  1. ... 82. 82.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]]


In [22]:

#3x3 Right disparity map
for x in range(1,row-1):     
    for y in range(1,col-1):
        block1 = view5[x-1:x+2,y-1:y+2] #block for view1
        min_dist = sys.maxsize    #max integer value 
        for z in range(y,y+100):#threshold : 100px
            if z < col-1:
                block2 = view1[x-1:x+2,z-1:z+2]
                result = np.sum((np.subtract(block1,block2)) ** 2)  
            if z > col-1:
                z = col-1
                
            if(result < min_dist):
                min_dist = result
                match_y = z 
                
                
        disp_5[x,y] = np.absolute(y - match_y) 



In [23]:
MSE_right = np.mean(np.square(disp5 - disp_5))#https://stackoverflow.com/questions/16774849/mean-squared-error-in-numpy
print("MSE Right:",MSE_right)


cv2.imshow('disp5_norm', disp_5/255)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imwrite('C:/Documents-Summer2018/CVIP/PA2/OPTnew/disparity_right_3x3.jpg',disp_5)


print('disp_5',disp_5)
print('view5',view5)


MSE Right: 407.7884887046874
disp_5 [[ 0.  0.  0. ...  0.  0.  0.]
 [ 0. 28. 27. ...  0.  0.  0.]
 [ 0. 25. 24. ...  0.  0.  0.]
 ...
 [ 0. 79. 79. ...  0.  0.  0.]
 [ 0. 79. 79. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]]
view5 [[ 56  56  55 ...  90  91  91]
 [ 59  58  57 ...  90  91  91]
 [ 55  54  56 ...  90  90  89]
 ...
 [146 144 144 ... 107 105 103]
 [150 145 145 ... 107 106 103]
 [152 150 149 ... 105 105 103]]


In [25]:
#consistency check

consistencyL = np.zeros((row,col))

#fill up consistency left by backprop

for x in range(1,row-1):
    for y in range(1,col-1):
        dispval_left = disp_1[x][y]
        if(y-dispval_left > 0):
            dispval_right = disp_5[x][int(abs(y-dispval_left))]
        else:
            dispval_right = disp_5[x][y]
        if(dispval_left == dispval_right):
            consistencyL[x][y] = dispval_left
        else:
            consistencyL[x][y] = 0


cv2.imshow('Consistency Left', consistencyL/255)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imwrite('C:/Documents-Summer2018/CVIP/PA2/OPTnew/Consistency_Left.jpg',consistencyL)
            
        

True

In [29]:
mse_cL = np.mean(np.square(disp1 - consistencyL))#https://stackoverflow.com/questions/16774849/mean-squared-error-in-numpy
print("MSE consistency left:",mse_cL)

MSE consistency left: 1302.631755297414


In [32]:
consistencyR = np.zeros((row,col))

for x in range(1,row-1):
    for y in range(1,col-1):
        dispval_right = disp_5[x][y]
        if(y+dispval_right < col-1):
            dispval_left = disp_1[x][int(y+dispval_right)]
        else:
            dispval_left=disp_1[x][y]
        if(dispval_right == dispval_left):
            consistencyR[x][y] = dispval_right
        else:
            consistencyR[x][y] = 0


cv2.imshow('Consistency Right', consistencyR/255)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imwrite('C:/Documents-Summer2018/CVIP/PA2/OPTnew/Consistency_Right.jpg',consistencyR)


            
        

True

In [33]:
mse_cR = np.mean(np.square(disp5 - consistencyR))#https://stackoverflow.com/questions/16774849/mean-squared-error-in-numpy
print("MSE consistency Right:",mse_cR)

MSE consistency Right: 1173.9941451170394
